In [9]:
%matplotlib notebook
%reload_ext autoreload
%autoreload 2

# Set defaults for charts so they aren't so tiny!
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [10,5]

import numpy
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from pprint import pprint
import pandas as pd
import csv
import sklearn.covariance
import sklearn.linear_model
import scipy.stats

In [10]:
with open('loss_results.csv', 'rt') as file:
    reader = csv.DictReader(file)
    results = list([dict(item) for item in reader])
    
results = [{key:float(item) for key,item in result.items()} for result in results]
for result in results:
    result['stdvsmedian'] = result['std'] / result['median']
    result['stdvsmin'] = result['std'] / result['min']

    
nonFeatureKeys=['gamma', 'loss', 'min', 'median', 'max']
lossKeys = ['adapt10', 'adapt25', 'adapt5', 'adapt50', 'exponential', 'identity', 'logarithmic', 'squareroot']
adaptKeys = ['adapt10', 'adapt25', 'adapt5', 'adapt50']

for result in results:
    best = min(lossKeys, key=lambda key: result[key])
    result['best'] = best

    adaptBest = min(adaptKeys, key=lambda key: result[key])
    result['bestAdapt'] = adaptBest
    result['adaptSize'] = int(adaptBest[5:])
    
    
features = [{key:value for key,value in result.items() if key not in nonFeatureKeys} for result in results]
pd.DataFrame(features)

,adapt10,adapt25,adapt5,adapt50,adaptSize,best,best/median,bestAdapt,exponential,identity,kurtosis,logarithmic,range,skew,squareroot,std,stdvsmedian,stdvsmin
0,0.090128,0.092803,0.091144,0.089989,50,adapt50,0.583151,adapt50,0.094024,0.090276,3.700796,0.093175,0.426252,2.010797,0.091070,0.141640,0.887024,1.568960
1,0.246910,0.242677,0.247769,0.241006,50,logarithmic,0.829719,adapt50,0.243060,0.240160,2.586510,0.239081,0.366168,1.561908,0.244468,0.050522,0.174435,0.210369
2,0.289552,0.294397,0.292206,0.288942,50,adapt50,0.887165,adapt50,0.294389,0.291604,3.173669,0.289012,0.404711,1.741472,0.291523,0.045272,0.137686,0.155252
3,0.089910,0.088122,0.084623,0.092991,5,adapt5,0.612115,adapt5,0.088701,0.091876,3.969323,0.089861,0.293783,1.822202,0.086796,0.072821,0.482384,0.792594
4,0.165860,0.166876,0.165137,0.164680,50,adapt50,0.769223,adapt50,0.168499,0.167179,2.247488,0.167927,0.318489,1.492068,0.167068,0.059712,0.272902,0.357174
5,0.224851,0.214256,0.214884,0.221498,25,adapt25,0.856231,adapt25,0.219828,0.218656,2.409897,0.219445,0.339768,1.622108,0.217058,0.050509,0.197583,0.230997
6,0.167884,0.162116,0.171609,0.171506,25,adapt25,0.654055,adapt25,0.174863,0.168980,1.620866,0.178092,0.372586,1.242629,0.177376,0.080045,0.310548,0.473695
7,0.271319,0.275238,0.270611,0.284243,5,adapt5,0.848013,adapt5,0.275847,0.274259,1.180774,0.273759,0.377926,1.224274,0.274380,0.044150,0.136428,0.160981
8,0.090576,0.089731,0.089766,0.089702,50,exponential,0.824718,adapt50,0.088950,0.089566,7.866427,0.090391,0.341156,2.661196,0.089662,0.089617,0.819111,1.000566
9,0.268284,0.277695,0.269728,0.275066,10,adapt10,0.860665,adapt10,0.277898,0.271180,2.036900,0.275050,0.372794,1.408121,0.274930,0.040548,0.128655,0.149523


In [11]:

counts = {}
adaptCounts = {}
adaptSizes = []
for key in lossKeys:
    counts[key] = 0
for key in adaptKeys:
    adaptCounts[key] = 0
for result in results:
    counts[result['best']] += 1
    adaptCounts[result['bestAdapt']] += 1
    adaptSizes.append(result['adaptSize'])

pprint(counts)

{'adapt10': 109,
 'adapt25': 103,
 'adapt5': 104,
 'adapt50': 119,
 'exponential': 110,
 'identity': 105,
 'logarithmic': 120,
 'squareroot': 96}


In [14]:
counts = {}
for key in lossKeys:
    counts[key] = 0
for key in adaptKeys:
    adaptCounts[key] = 0
for result in results:
    for key in lossKeys:
        if result[key] > result['identity']:
            counts[key] += 1
for key in lossKeys:
    counts[key] = counts[key] / len(results)
    
pprint(counts)

{'adapt10': 0.48036951501154734,
 'adapt25': 0.5127020785219399,
 'adapt5': 0.5057736720554272,
 'adapt50': 0.4953810623556582,
 'exponential': 0.5,
 'identity': 0.0,
 'logarithmic': 0.49191685912240185,
 'squareroot': 0.4930715935334873}


In [17]:
averages = {}
for key in lossKeys:
    values = [result[key] for result in results]
    averages[key] = numpy.mean(values)*1

pprint(averages)


{'adapt10': 0.20638841049856826,
 'adapt25': 0.20653561869378104,
 'adapt5': 0.2064733937191374,
 'adapt50': 0.2064599203267815,
 'exponential': 0.2065021215922073,
 'identity': 0.20652949885776323,
 'logarithmic': 0.2063690821730577,
 'squareroot': 0.2064465462948599}
